 <center>JCAFB-2025 - Cadastro de Pacientes Off-Line<center>
 <center>Campanha DHC (2) - 2025<center>

# Exportar a lista de todos os Pacientes cadastrados (Referência)

1. Conectar-se, via *browser*, ao *Odoo* do servidor [clvheatlh-jcafb-2025n-aws-pro](https://34.224.24.169).

1. Acessar a *View* *Patients*:
    - Menu de acesso:
        - **Health** » **Health** » **Patient** » **Patients**

1. Selecionar todos os Pacientes.

1. Executar a Ação "**Exportar**" (**852**):
    - Parâmetros utilizados:
        - *Quero atualizar os dados (compatibilidade com importação e exportação)*: "**marcado**"
        - *Campos para exportar*: » *Modelo*: "**jcafb_2025_cadastro_pacientes_dhc2_ref**"
    1. Utilize o botão **Exportar** para executar a Ação.


# Parâmetros globais

In [1]:
import pandas as pd
import re


In [2]:
import logging

logging.basicConfig(level=logging.DEBUG)
_logger = logging.getLogger(__name__)


## Cadastro de Referência

In [3]:
jcafb_cadastro_pacientes_ref_path = 'data/jcafb_2025_cadastro_pacientes_dhc2_ref.xlsx'
jcafb_cadastro_pacientes_ref_sheet = 'cadastro_pacientes_dhc2_ref'

## Cadastros de Campanha

### Cadastro de Campanha (arquivo 1)

In [4]:
jcafb_cadastro_campanha_1_path = 'data/jcafb_2025_cadastro_pacientes_dhc2_a1.xlsx'
jcafb_cadastro_campanha_1_sheet = 'cadastro_pacientes_dhc2_a1'

### Cadastro de Campanha (arquivo 2)

In [5]:
jcafb_cadastro_campanha_2_path = 'data/jcafb_2025_cadastro_pacientes_dhc2_a2.xlsx'
jcafb_cadastro_campanha_2_sheet = 'cadastro_pacientes_dhc2_a2'

### Cadastro de Campanha (arquivo 3)

In [6]:
jcafb_cadastro_campanha_3_path = 'data/jcafb_2025_cadastro_pacientes_dhc2_a3.xlsx'
jcafb_cadastro_campanha_3_sheet = 'cadastro_pacientes_dhc2_a3'

### Cadastro de Campanha (arquivo 4)

In [7]:
jcafb_cadastro_campanha_4_path = 'data/jcafb_2025_cadastro_pacientes_dhc2_a4.xlsx'
jcafb_cadastro_campanha_4_sheet = 'cadastro_pacientes_dhc2_a4'

## Arquivos de Saída

### Lista de Códigos Inválidos

In [8]:
invalid_codes_path = 'data/jcafb_2025_dhc2_invalid_codes.xlsx'
invalid_codes_sheet = 'dhc2_invalid_codes'

### Lista de Ids incluídos

In [9]:
new_ids_path = 'data/jcafb_2025_dhc2_new_ids.xlsx'
new_ids_sheet = 'dhc2_new_ids'

### Lista de Registros incluídos

In [10]:
new_records_path = 'data/jcafb_2025_dhc2_new_records.xlsx'
new_records_sheet = 'dhc2_new_records'

### Lista de Atualizações

In [11]:
updt_records_path = 'data/jcafb_2025_dhc2_updt_records.xlsx'
updt_records_sheet = 'dhc2_updt_records'

### Cadastro de Pacientes Atualizado

In [12]:
jcafb_cadastro_pacientes_updt_path = 'data/jcafb_2025_cadastro_pacientes_dhc2_updt.xlsx'
jcafb_cadastro_pacientes_updt_sheet = 'cadastro_pacientes_dhc2_updt'

### Cadastro de Pacientes para importação no Odoo

In [13]:
jcafb_cadastro_pacientes_odoo_path = 'data//jcafb_2025_cadastro_pacientes_dhc2_odoo.xlsx'
jcafb_cadastro_pacientes_odoo_sheet = 'cadastro_pacientes_dhc2_odoo'

# Funções de Suporte

In [14]:
def somaPoderada(numero):
    i = 0
    soma = 0
    while i<len(numero):
        soma = soma+int(numero[i]) * (15 - i)
        i = i+1
    return soma

def validaCNS(numero):
    numero = str(numero)
    if numero.isdigit():
        if re.match(r'[1-2]\d{10}00[0-1]\d$',numero) or re.match(r'[7-9]\d{14}$',numero):
            return somaPoderada(numero) % 11 == 0
    return False

# validaCNS('703008820145471')
# validaCNS('702803658947067')
# validaCNS('704301554368493')
# validaCNS('700506784991850')


In [15]:
def format_code(code_seq):
    code = list(map(int, str(code_seq)))
    code_len = len(code)
    while len(code) < 14:
        code.insert(0, 0)
    while len(code) < 16:
        n = sum([(len(code) + 1 - i) * v for i, v in enumerate(code)]) % 11
        if n > 1:
            f = 11 - n
        else:
            f = 0
        code.append(f)
    code_str = "%s.%s.%s.%s.%s-%s" % (str(code[0]) + str(code[1]),
                                      str(code[2]) + str(code[3]) + str(code[4]),
                                      str(code[5]) + str(code[6]) + str(code[7]),
                                      str(code[8]) + str(code[9]) + str(code[10]),
                                      str(code[11]) + str(code[12]) + str(code[13]),
                                      str(code[14]) + str(code[15]))
    if code_len <= 3:
        code_form = code_str[18 - code_len:21]
    elif code_len > 3 and code_len <= 6:
        code_form = code_str[17 - code_len:21]
    elif code_len > 6 and code_len <= 9:
        code_form = code_str[16 - code_len:21]
    elif code_len > 9 and code_len <= 12:
        code_form = code_str[15 - code_len:21]
    elif code_len > 12 and code_len <= 14:
        code_form = code_str[14 - code_len:21]
    return code_form

def validaCPF(code):

    if code is not False:

        sequence_str = ''
        for c in code:
            if c.isdigit():
                sequence_str = sequence_str + c
        sequence_str = sequence_str[:len(sequence_str) - 2]
        _format_code = format_code(sequence_str)

        if code == _format_code:
            return True

    return False

# validaCPF('551.583.048-23')
# validaCPF('525.338.988-13')

# Leitura do Cadastro de Referência

In [16]:
xls_jcafb_cadastro_pacientes_ref = pd.ExcelFile(jcafb_cadastro_pacientes_ref_path)
jcafb_cadastro_pacientes_ref = pd.read_excel(xls_jcafb_cadastro_pacientes_ref,
                                             sheet_name=jcafb_cadastro_pacientes_ref_sheet,
                                             na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_ref Columns:', jcafb_cadastro_pacientes_ref.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_ref Number of rows:', len(jcafb_cadastro_pacientes_ref))


INFO:__main__:--> jcafb_cadastro_pacientes_ref Columns: ['id', 'name', 'code', 'birthday', 'gender', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'employee_id/name', 'category_names', 'DHC1', 'Anemia1', 'DHC2', 'Anemia2']
INFO:__main__:--> jcafb_cadastro_pacientes_ref Number of rows: 852


# Leitura do Cadastro Atualizado

In [17]:
xls_jcafb_cadastro_pacientes_updt = pd.ExcelFile(jcafb_cadastro_pacientes_ref_path)
jcafb_cadastro_pacientes_updt = pd.read_excel(xls_jcafb_cadastro_pacientes_updt,
                                              sheet_name=jcafb_cadastro_pacientes_ref_sheet,
                                              na_filter=False)

pos = len(jcafb_cadastro_pacientes_updt.columns)
jcafb_cadastro_pacientes_updt.insert(pos, 'revised', '')
jcafb_cadastro_pacientes_updt.insert(pos, 'new', '')

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_ref Columns:', jcafb_cadastro_pacientes_updt.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_ref Number of rows:', len(jcafb_cadastro_pacientes_updt))


INFO:__main__:--> jcafb_cadastro_pacientes_ref Columns: ['id', 'name', 'code', 'birthday', 'gender', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'employee_id/name', 'category_names', 'DHC1', 'Anemia1', 'DHC2', 'Anemia2', 'new', 'revised']
INFO:__main__:--> jcafb_cadastro_pacientes_ref Number of rows: 852


# Leitura dos Cadastros de Campanha

## Cadastro de Campanha (arquivo 1)

In [18]:
xls_jcafb_cadastro_campanha_1 = pd.ExcelFile(jcafb_cadastro_campanha_1_path)
jcafb_cadastro_campanha_1 = pd.read_excel(xls_jcafb_cadastro_campanha_1,
                                          sheet_name=jcafb_cadastro_campanha_1_sheet,
                                          na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_1 Columns:', jcafb_cadastro_campanha_1.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_1 Number of rows:', len(jcafb_cadastro_campanha_1))


INFO:__main__:--> jcafb_cadastro_campanha_1 Columns: ['id', 'name', 'code', 'birthday', 'gender', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'employee_id/name', 'category_names', 'DHC1', 'Anemia1', 'DHC2', 'Anemia2']
INFO:__main__:--> jcafb_cadastro_campanha_1 Number of rows: 852


## Cadastro de Campanha (arquivo 2)

In [19]:
xls_jcafb_cadastro_campanha_2 = pd.ExcelFile(jcafb_cadastro_campanha_2_path)
jcafb_cadastro_campanha_2 = pd.read_excel(xls_jcafb_cadastro_campanha_2,
                                          sheet_name=jcafb_cadastro_campanha_2_sheet,
                                          na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_2 Columns:', jcafb_cadastro_campanha_2.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_2 Number of rows:', len(jcafb_cadastro_campanha_2))


INFO:__main__:--> jcafb_cadastro_campanha_2 Columns: ['id', 'name', 'code', 'birthday', 'gender', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'employee_id/name', 'category_names', 'DHC1', 'Anemia1', 'DHC2', 'Anemia2']
INFO:__main__:--> jcafb_cadastro_campanha_2 Number of rows: 852


## Cadastro de Campanha (arquivo 3)

In [20]:
xls_jcafb_cadastro_campanha_3 = pd.ExcelFile(jcafb_cadastro_campanha_3_path)
jcafb_cadastro_campanha_3 = pd.read_excel(xls_jcafb_cadastro_campanha_3,
                                          sheet_name=jcafb_cadastro_campanha_3_sheet,
                                          na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_3 Columns:', jcafb_cadastro_campanha_3.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_3 Number of rows:', len(jcafb_cadastro_campanha_3))


INFO:__main__:--> jcafb_cadastro_campanha_3 Columns: ['id', 'name', 'code', 'birthday', 'gender', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'employee_id/name', 'category_names', 'DHC1', 'Anemia1', 'DHC2', 'Anemia2']
INFO:__main__:--> jcafb_cadastro_campanha_3 Number of rows: 852


## Cadastro de Campanha (arquivo 4)

In [21]:
xls_jcafb_cadastro_campanha_4 = pd.ExcelFile(jcafb_cadastro_campanha_4_path)
jcafb_cadastro_campanha_4 = pd.read_excel(xls_jcafb_cadastro_campanha_4,
                                          sheet_name=jcafb_cadastro_campanha_4_sheet,
                                          na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_4 Columns:', jcafb_cadastro_campanha_4.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_campanha_4 Number of rows:', len(jcafb_cadastro_campanha_4))


INFO:__main__:--> jcafb_cadastro_campanha_4 Columns: ['id', 'name', 'code', 'birthday', 'gender', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'employee_id/name', 'category_names', 'DHC1', 'Anemia1', 'DHC2', 'Anemia2']
INFO:__main__:--> jcafb_cadastro_campanha_4 Number of rows: 852


# Validar Cadastros de Campanha

## Validação do "Código do Paciente"

In [22]:
column_names = ['íd', 'name', 'code', 'source', 'row']
invalid_codes = pd.DataFrame(columns=column_names)

def code_validation(jcafb_cadastro_campanha, source):

    for i, row in jcafb_cadastro_campanha.iterrows():

        _code = row['code']
        validate = False
        if validaCNS(_code):
            validate = True
        if not validate:
            if validaCPF(_code):
                validate = True
        if not validate:
            new_row = {'id': row['id'],
                       'name': row['name'],
                       'code': _code,
                       'row': i + 2,
                       'source': source
                      }
            _logger.info(u'%s %s %s', '-->', 'Invalid Code:', new_row)
            invalid_codes.loc[len(invalid_codes)] = new_row


### Cadastro de Campanha (arquivo 1)

In [23]:
code_validation(jcafb_cadastro_campanha_1, jcafb_cadastro_campanha_1_sheet)


### Cadastro de Campanha (arquivo 2)

In [24]:
code_validation(jcafb_cadastro_campanha_2, jcafb_cadastro_campanha_2_sheet)


### Cadastro de Campanha (arquivo 3)

In [25]:
code_validation(jcafb_cadastro_campanha_3, jcafb_cadastro_campanha_3_sheet)


### Cadastro de Campanha (arquivo 4)

In [26]:
code_validation(jcafb_cadastro_campanha_4, jcafb_cadastro_campanha_4_sheet)


## Lista de Códigos Inválidos

In [27]:
invalid_codes

,íd,name,code,source,row


## Inclusão de novos "ids"

In [28]:
column_names = ['id', 'name', 'code', 'source', 'row']
new_ids = pd.DataFrame(columns=column_names)

def new_ids_inclusion(jcafb_cadastro_campanha, source):

    for i, row in jcafb_cadastro_campanha.iterrows():

        _id = row['id']
        _code = row['code']

        if _id == '':
            sequence_str = ''
            for c in _code:
                if c.isdigit():
                    sequence_str = sequence_str + c
            _id = '__import__.clv_patient_' + sequence_str
            new_row = {'id': _id,
                       'name': row['name'],
                       'code': _code,
                       'source': source,
                       'row': i + 2
                      }
            _logger.info(u'%s %s %s', '-->', 'Novo id:', new_row)
            new_ids.loc[len(new_ids)] = new_row
            jcafb_cadastro_campanha['id'].values[i] = _id


### Cadastro de Campanha (arquivo 1)

In [29]:
new_ids_inclusion(jcafb_cadastro_campanha_1, jcafb_cadastro_campanha_1_sheet)


### Cadastro de Campanha (arquivo 2)

In [30]:
new_ids_inclusion(jcafb_cadastro_campanha_2, jcafb_cadastro_campanha_2_sheet)


### Cadastro de Campanha (arquivo 3)

In [31]:
new_ids_inclusion(jcafb_cadastro_campanha_3, jcafb_cadastro_campanha_3_sheet)


### Cadastro de Campanha (arquivo 4)

In [32]:
new_ids_inclusion(jcafb_cadastro_campanha_4, jcafb_cadastro_campanha_4_sheet)


## Lista de Ids incluídos

In [33]:
new_ids

,id,name,code,source,row


# Atualizar Cadastro

## Incluir novos registros

In [34]:
column_names = ['id', 'name', 'code', 'source', 'source_row', 'updt_row']
new_records = pd.DataFrame(columns=column_names)

def new_records_inclusion(jcafb_cadastro_campanha, source):

    for i, row in jcafb_cadastro_campanha.iterrows():

        _id = row['id']
        index = jcafb_cadastro_pacientes_ref.loc[jcafb_cadastro_pacientes_ref['id'] == _id]
        if len(index) == 0:
            # new_row = {'id': _id,
            #            'name': row['name'],
            #            'code': row['code'],
            #            'birthday': row['birthday'],
            #            'gender': row['gender'],
            #           }
            new_record_pos = len(jcafb_cadastro_pacientes_updt)
            # jcafb_cadastro_pacientes_updt.loc[new_record_pos] = new_row
            jcafb_cadastro_pacientes_updt.loc[new_record_pos] = row
            jcafb_cadastro_pacientes_updt['new'].values[new_record_pos] = 'x'
            _logger.info(u'%s %s %s', '-->', 'Novo Registro:', jcafb_cadastro_pacientes_updt.values[new_record_pos])

            new_row = {'id': _id,
                       'name': row['name'],
                       'code': row['code'],
                       'source': source,
                       'source_row': i + 2,
                       'updt_row': new_record_pos + 2
                      }
            pos = len(new_records)
            new_records.loc[pos] = new_row


### Cadastro de Campanha (arquivo 1)

In [35]:
new_records_inclusion(jcafb_cadastro_campanha_1, jcafb_cadastro_campanha_1_sheet)


### Cadastro de Campanha (arquivo 2)

In [36]:
new_records_inclusion(jcafb_cadastro_campanha_2, jcafb_cadastro_campanha_2_sheet)


### Cadastro de Campanha (arquivo 3)

In [37]:
new_records_inclusion(jcafb_cadastro_campanha_3, jcafb_cadastro_campanha_3_sheet)


### Cadastro de Campanha (arquivo 4)

In [38]:
new_records_inclusion(jcafb_cadastro_campanha_4, jcafb_cadastro_campanha_4_sheet)


## Lista de Registros incluídos

In [39]:
new_records

,id,name,code,source,source_row,updt_row


## Cadastro de Pacientes Atualizado

In [40]:
print(jcafb_cadastro_pacientes_updt)

                                     id                            name  \
0    __import__.clv_patient_70905207815              Abel Lucas Bertoni   
1    __import__.clv_patient_82625760800              Adalberto de Sousa   
2    __import__.clv_patient_28019727892      Adelaide Francisco Machado   
3    __import__.clv_patient_05852199826        Adelia Aparecida Fontana   
4    __import__.clv_patient_04227049890          Ademar Antonio Dagoano   
..                                  ...                             ...   
847  __import__.clv_patient_09614270861      Zeni Maria de Souza Pivato   
848  __import__.clv_patient_34700464844  Zenilce de Souza Lima da Silva   
849  __import__.clv_patient_82586993853          Zetir Maria Souza Tosi   
850  __import__.clv_patient_25732126880            Zilda Pereira Cruaia   
851  __import__.clv_patient_04980187846           Zilda Teixeira Cintra   

               code   birthday  gender        zip  \
0    709.052.078-15 1955-06-01    Male  16680-

## Atualizar Registros existentes

### Cadastro de Campanha (arquivo 1)

In [41]:
column_names = ['id', 'name', 'code', 'column_name', 'from', 'to', 'source', 'source_row']
updt_records = pd.DataFrame(columns=column_names)

def new_records_inclusion(jcafb_cadastro_campanha, source):

    # column_names_list = ['name', 'birthday', 'gender',
    #                      'zip', 'street_name', 'street_number', 'street_number2', 'street2',
    #                      'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id',
    #                      'phone', 'notes',
    #                      'DHC1', 'Anemia1', 'DHC2', 'Anemia2']
    column_names_list = ['name', 'birthday', 'gender',
                         'zip', 'street_name', 'street_number', 'street_number2', 'street2',
                         'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id',
                         'phone',
                         'DHC1', 'Anemia1', 'DHC2', 'Anemia2']

    for i, row in jcafb_cadastro_campanha.iterrows():

        _id = row['id']
        index = jcafb_cadastro_pacientes_ref.loc[jcafb_cadastro_pacientes_ref['id'] == _id]
        revised = False
        if len(index) > 0:
            for column_name in column_names_list:
                if row[column_name] != index[column_name].iloc[0]:
                    jcafb_cadastro_pacientes_updt.loc[jcafb_cadastro_pacientes_updt['id'] == _id, column_name] = row[column_name]
                    revised = True

                    new_row = {'id': _id,
                               'name': row['name'],
                               'code': row['code'],
                               'column_name': column_name,
                               'from': index[column_name].iloc[0],
                               'to': row[column_name],
                               'source': source,
                               'source_row': i + 2
                              }
                    pos = len(updt_records)
                    updt_records.loc[pos] = new_row
                    _logger.info(u'%s %s %s', '-->', 'Atualização:', new_row)

            if revised:
                jcafb_cadastro_pacientes_updt.loc[jcafb_cadastro_pacientes_updt['id'] == _id, 'revised'] = 'x'


In [42]:
new_records_inclusion(jcafb_cadastro_campanha_1, jcafb_cadastro_campanha_1_sheet)


### Cadastro de Campanha (arquivo 2)

In [43]:
new_records_inclusion(jcafb_cadastro_campanha_2, jcafb_cadastro_campanha_2_sheet)


### Cadastro de Campanha (arquivo 3)

In [44]:
new_records_inclusion(jcafb_cadastro_campanha_3, jcafb_cadastro_campanha_3_sheet)


### Cadastro de Campanha (arquivo 4)

In [45]:
new_records_inclusion(jcafb_cadastro_campanha_4, jcafb_cadastro_campanha_1_sheet)


## Lista de Atualizações

In [46]:
updt_records

,id,name,code,column_name,from,to,source,source_row


## Cadastro de Pacientes Atualizado

In [47]:
 print(jcafb_cadastro_pacientes_updt)

                                     id                            name  \
0    __import__.clv_patient_70905207815              Abel Lucas Bertoni   
1    __import__.clv_patient_82625760800              Adalberto de Sousa   
2    __import__.clv_patient_28019727892      Adelaide Francisco Machado   
3    __import__.clv_patient_05852199826        Adelia Aparecida Fontana   
4    __import__.clv_patient_04227049890          Ademar Antonio Dagoano   
..                                  ...                             ...   
847  __import__.clv_patient_09614270861      Zeni Maria de Souza Pivato   
848  __import__.clv_patient_34700464844  Zenilce de Souza Lima da Silva   
849  __import__.clv_patient_82586993853          Zetir Maria Souza Tosi   
850  __import__.clv_patient_25732126880            Zilda Pereira Cruaia   
851  __import__.clv_patient_04980187846           Zilda Teixeira Cintra   

               code   birthday  gender        zip  \
0    709.052.078-15 1955-06-01    Male  16680-

# Exportar os arquivos

## Exportar Lista de Códigos Inválidos

In [48]:
invalid_codes.to_excel(invalid_codes_path, sheet_name=invalid_codes_sheet, index=False)
_logger.info(u'%s %s %s', '-->', 'invalid_codes:', invalid_codes_path)

INFO:__main__:--> invalid_codes: data/jcafb_2025_dhc2_invalid_codes.xlsx


## Exportar Lista de Ids incluídos

In [49]:
new_ids.to_excel(new_ids_path, sheet_name=new_ids_sheet, index=False)
_logger.info(u'%s %s %s', '-->', 'new_records:', new_ids_path)

INFO:__main__:--> new_records: data/jcafb_2025_dhc2_new_ids.xlsx


## Exportar a Lista de Registros incluídos

In [50]:
new_records.to_excel(new_records_path, sheet_name=new_records_sheet, index=False)
_logger.info(u'%s %s %s', '-->', 'new_records:', new_records_path)

INFO:__main__:--> new_records: data/jcafb_2025_dhc2_new_records.xlsx


## Exportar a Lista de Atualizações

In [51]:
updt_records.to_excel(updt_records_path, sheet_name=updt_records_sheet, index=False)
_logger.info(u'%s %s %s', '-->', 'new_records:', updt_records_path)

INFO:__main__:--> new_records: data/jcafb_2025_dhc2_updt_records.xlsx


## Exportar o Cadastro de Pacientes Atualizado

In [52]:
jcafb_cadastro_pacientes_updt.to_excel(jcafb_cadastro_pacientes_updt_path,
                                       sheet_name=jcafb_cadastro_pacientes_updt_sheet, index=False)
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_updt:', jcafb_cadastro_pacientes_updt_path)
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_updt Columns:', jcafb_cadastro_pacientes_updt.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_updt Number of rows:', len(jcafb_cadastro_pacientes_updt))


INFO:__main__:--> jcafb_cadastro_pacientes_updt: data/jcafb_2025_cadastro_pacientes_dhc2_updt.xlsx
INFO:__main__:--> jcafb_cadastro_pacientes_updt Columns: ['id', 'name', 'code', 'birthday', 'gender', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'employee_id/name', 'category_names', 'DHC1', 'Anemia1', 'DHC2', 'Anemia2', 'new', 'revised']
INFO:__main__:--> jcafb_cadastro_pacientes_updt Number of rows: 852


## Importar "*Patient Markers*"

1. Conectar-se, via *browser*, ao *Odoo* do servidor [clvheatlh-jcafb-2025n-aws-pro](https://34.224.24.169).

1. Acessar a *View* *Global Tags*:
    - Menu de acesso:
        - **Health** » **Configuration** » **Patient** » **Markers**

1. Executar a importação do arquivo "**/opt/clvsol/clvhealth_jcafb_2025_15/project/data/jcafb_2025_clv_patient_marker.xlsx**":

    - Menu de acesso:
        - **Favoritos** » **Importar registros**

    1. Utilizar o botão **Upload File**.

    1. Selecionar o arquivo "**/opt/clvsol/clvhealth_jcafb_2025_15/project/data/jcafb_2025_clv_patient_marker.xlsx**".

    1. Utilize o botão **Importar** para executar a Ação.



In [53]:
clv_patient_marker_path = 'data/jcafb_2025_clv_patient_marker.xlsx'
clv_patient_marker_sheet = 'clv_patient_marker'

xls_clv_patient_marker = pd.ExcelFile(clv_patient_marker_path)
clv_patient_marker = pd.read_excel(xls_clv_patient_marker, sheet_name=clv_patient_marker_sheet, na_filter=False)

_logger.info(u'%s %s %s', '-->', 'clv_patient_marker Columns:', clv_patient_marker.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'clv_patient_marker Number of rows:', len(clv_patient_marker))

clv_patient_marker

INFO:__main__:--> clv_patient_marker Columns: ['id', 'name', 'description']
INFO:__main__:--> clv_patient_marker Number of rows: 4


,id,name,description
0,__import__.clv_patient_marker_DHC1,DHC1,Campanha DHC 1
1,__import__.clv_patient_marker_Anemia1,Anemia1,Campanha Anemia 1
2,__import__.clv_patient_marker_DHC2,DHC2,Campanha DHC 2
3,__import__.clv_patient_marker_Anemia2,Anemia2,Campanha Anemia 2


## Recarregar o Cadastro de Pacientes Atualizado

In [54]:
xls_jcafb_cadastro_pacientes_updt = pd.ExcelFile(jcafb_cadastro_pacientes_updt_path)
jcafb_cadastro_pacientes_updt = pd.read_excel(xls_jcafb_cadastro_pacientes_updt,
                                              sheet_name=jcafb_cadastro_pacientes_updt_sheet,
                                              na_filter=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_updt Columns:', jcafb_cadastro_pacientes_updt.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_updt Number of rows:', len(jcafb_cadastro_pacientes_updt))


INFO:__main__:--> jcafb_cadastro_pacientes_updt Columns: ['id', 'name', 'code', 'birthday', 'gender', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id', 'phone', 'state', 'employee_id/name', 'category_names', 'DHC1', 'Anemia1', 'DHC2', 'Anemia2', 'new', 'revised']
INFO:__main__:--> jcafb_cadastro_pacientes_updt Number of rows: 852


## Exportar o Cadastro de Pacientes para importação pelo Odoo

In [55]:
# jcafb_cadastro_odoo_path = 'data/jcafb_cadastro_odoo.xlsx'
# jcafb_cadastro_odoo_sheet = 'Sheet1'
jcafb_cadastro_pacientes_odoo_path = 'data//jcafb_2025_cadastro_pacientes_dhc2_odoo.xlsx'
jcafb_cadastro_pacientes_odoo_sheet = 'cadastro_pacientes_dhc2_odoo'

# column_names = ['id', 'name', 'code', 'birthday', 'gender',
#                 'zip', 'street_name', 'street_number', 'street_number2', 'street2',
#                 'city_id/name', 'city_id/id', 'state_id/name', 'state_id/id', 'country_id/name', 'country_id/id',
#                 'phone',
#                 'state', 'employee_id/name', 'category_names',
#                 'marker_ids/id,
#                 'DHC2', 'Anemia1', 'DHC2', 'Anemia2',
#                 'new', 'revised']
column_names = ['id', 'name', 'code', 'birthday', 'gender',
                'zip', 'street_name', 'street_number', 'street_number2', 'street2',
                'city_id/id', 'state_id/id', 'country_id/id',
                'phone',
                'notes', 'marker_ids/id']

jcafb_cadastro_pacientes_odoo = pd.DataFrame(columns=column_names)

for i, row in jcafb_cadastro_pacientes_updt.iterrows():

    phase_id = 'JCAFB-2025'

    marker_ids = ''
    if row['DHC1'] == 'x':
        if marker_ids == '':
            marker_ids = '__import__.clv_patient_marker_DHC1'
        else:
            marker_ids = marker_ids +  ',__import__.clv_patient_marker_DHC1'
    if row['Anemia1'] == 'x':
        if marker_ids == '':
            marker_ids = '__import__.clv_patient_marker_Anemia1'
        else:
            marker_ids = marker_ids +  ',__import__.clv_patient_marker_Anemia1'
    if row['DHC2'] == 'x':
        if marker_ids == '':
            marker_ids = '__import__.clv_patient_marker_DHC2'
        else:
            marker_ids = marker_ids +  ',__import__.clv_patient_marker_DHC2'
    if row['Anemia2'] == 'x':
        if marker_ids == '':
            marker_ids = '__import__.clv_patient_marker_Anemia2'
        else:
            marker_ids = marker_ids +  ',__import__.clv_patient_marker_Anemia2'

    if row['new'] == 'x' or row['revised'] == 'x':
        new_row = {'id': row['id'],
                   'name': row['name'],
                   'code': row['code'],
                   'birthday': row['birthday'],
                   'gender': row['gender'],
                   'zip': row['zip'],
                   'street_name': row['street_name'],
                   'street_number': row['street_number'],
                   'street_number2': row['street_number2'],
                   'street2': row['street2'],
                   'city_id/id': row['city_id/id'],
                   'state_id/id': row['state_id/id'],
                   'country_id/id': row['country_id/id'],
                   'phone': row['phone'],
                   'phase_id': phase_id,
                   'marker_ids/id': marker_ids
                  }
        jcafb_cadastro_pacientes_odoo.loc[len(jcafb_cadastro_pacientes_odoo)] = new_row

jcafb_cadastro_pacientes_odoo.to_excel(jcafb_cadastro_pacientes_odoo_path,
                                       sheet_name=jcafb_cadastro_pacientes_odoo_sheet,
                                       index=False)

_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_odoo:', jcafb_cadastro_pacientes_odoo_path)
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_odoo Columns:', jcafb_cadastro_pacientes_odoo.columns.tolist())
_logger.info(u'%s %s %s', '-->', 'jcafb_cadastro_pacientes_odoo Number of rows:', len(jcafb_cadastro_pacientes_odoo))


INFO:__main__:--> jcafb_cadastro_pacientes_odoo: data//jcafb_2025_cadastro_pacientes_dhc2_odoo.xlsx
INFO:__main__:--> jcafb_cadastro_pacientes_odoo Columns: ['id', 'name', 'code', 'birthday', 'gender', 'zip', 'street_name', 'street_number', 'street_number2', 'street2', 'city_id/id', 'state_id/id', 'country_id/id', 'phone', 'notes', 'marker_ids/id']
INFO:__main__:--> jcafb_cadastro_pacientes_odoo Number of rows: 0


# Importar o Cadastro de Pacientes atualizado de Avaí

**Nota**:  <span style='color:green '> Repetir duas vezes o procedimento (confirmar) </span>

1. Conectar-se, via *browser*, ao *Odoo* do servidor [clvheatlh-jcafb-2025n-aws-pro](https://34.224.24.169).

1. Acessar a *View* *Patients*:
    - Menu de acesso:
        - **Health** » **Health** » **Patient** » **Patients**

1. Executar a importação do arquivo "**/opt/clvsol/clvhealth_jcafb_2025_15/project/data/jcafb_2025_cadastro_pacientes_dhc2_odoo.xlsx**":

    - Menu de acesso:
        - **Favoritos** » **Importar registros**

    1. Utilizar o botão **Upload File**.

    1. Selecionar o arquivo "**/opt/clvsol/clvhealth_jcafb_2025_15/project/data/jcafb_2025_cadastro_pacientes_dhc2_odoo.xlsx**".

    1. Utilize o botão **Importar** para executar a Ação.

